In [3]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv

In [4]:
import pandas as pd
from google.cloud import bigquery

client = bigquery.Client()

In [5]:
# data = pd.io.stata.read_stata('../../datasets/atlas_harvard/country_partner_hsproduct6digit_year_2021.dta')
hs_class = pd.io.stata.read_stata('../../datasets/atlas_harvard/hs_product.dta')

# product classification

In [14]:
hs_class["level"].unique()

array(['section', '2digit', '4digit', '6digit'], dtype=object)

In [15]:
hs_class.set_index("level").loc["section"]

,product_id,hs_product_code,hs_product_name_short_en,parent_id
level,,,,
section,0,0,Textiles,NaN
section,1,1,Agriculture,NaN
section,2,2,Stone,NaN
section,3,3,Minerals,NaN
section,4,4,Metals,NaN
section,5,5,Chemicals,NaN
section,6,6,Vehicles,NaN
section,7,7,Machinery,NaN
section,8,8,Electronics,NaN


In [8]:
hs_2digit = hs_class.set_index("level").loc["2digit"].set_index("hs_product_code")

In [9]:
len(hs_2digit)

102

In [52]:
hs_4digit = hs_class.set_index("level").loc["4digit"].set_index("hs_product_code")

hs_6digit = hs_class.set_index("level").loc["6digit"].set_index("hs_product_code")

hs_2digit.index.name = "hs_2digit"
hs_4digit.index.name = "hs_4digit"
hs_6digit.index.name = "hs_6digit"

hs_2digit.rename(columns={"hs_product_name_short_en": "hs_product_name_2digit"}, inplace=True)
hs_4digit.rename(columns={"hs_product_name_short_en": "hs_product_name_4digit"}, inplace=True)
hs_6digit.rename(columns={"hs_product_name_short_en": "hs_product_name_6digit"}, inplace=True)

In [51]:
hs_2digit.sample(10)

,product_id,hs_product_name_2digit,parent_id
hs_2digit,,,
95,193,Toys,7.0
47,146,Pulp of wood,1.0
28,127,Inorganic chemicals,5.0
08,107,Fruits and nuts,1.0
14,113,Other vegetable materials,1.0
90,188,"Apparatuses (optical, medical, etc.)",7.0
06,105,Plants,1.0
64,163,Footwear,0.0
23,122,Food residues and animal feed,1.0


In [57]:
hs_4digit.tail(10)

,product_id,hs_product_name_4digit,parent_id
hs_4digit,,,
9704,1888,Postage or revenue stamps,195.0
9705,1889,Collectors pieces,195.0
9706,1890,Antiques >100 years,195.0
9999,1891,Commodities not specified according to kind,196.0
XXXX,1892,Trade data discrepancies,196.0
unspecified,4000,Unspecified,400.0
travel,4001,Travel and tourism,401.0
transport,4002,Transport,402.0
ict,4003,ICT,403.0


In [58]:
hs_6digit.tail(10)

,product_id,hs_product_name_6digit,parent_id,hs_4digit,hs_2digit
hs_6digit,,,,,
970500,10035,Collections & collectors pieces,1889.0,9705,97
970600,10036,Antiques,1890.0,9706,97
999999,10037,"Commodities not specified, according to kind",1891.0,9999,99
9999AA,10038,Commodities not specified according to kind,1891.0,9999,99
XXXXXX,10039,Trade data discrepancies,1892.0,XXXX,XX
unspecified,11000,Unspecified,4000.0,unsp,un
travel,11001,Travel and tourism,4001.0,trav,tr
transport,11002,Transport,4002.0,tran,tr
ict,11003,ICT,4003.0,ict,ic


In [62]:
others = ["unspecified", "travel", "transport", "ict", "financial"]

hs_6digit.loc[:, "hs_4digit"] = hs_6digit.apply(lambda r: r.name if r.name in others else r.name[0:4], axis=1)
hs_6digit.loc[:, "hs_2digit"] = hs_6digit.apply(lambda r: r.name if r.name in others else r.name[0:2], axis=1)

In [63]:
hs_mapping = pd.merge(
    hs_6digit.reset_index(),
    hs_4digit[["hs_product_name_4digit"]],
    how="outer",
    on="hs_4digit")

hs_mapping = pd.merge(
    hs_mapping,
    hs_2digit[["hs_product_name_2digit"]],
    how="outer",
    on="hs_2digit")

In [64]:
hs_6digit.tail(10)

,product_id,hs_product_name_6digit,parent_id,hs_4digit,hs_2digit
hs_6digit,,,,,
970500,10035,Collections & collectors pieces,1889.0,9705,97
970600,10036,Antiques,1890.0,9706,97
999999,10037,"Commodities not specified, according to kind",1891.0,9999,99
9999AA,10038,Commodities not specified according to kind,1891.0,9999,99
XXXXXX,10039,Trade data discrepancies,1892.0,XXXX,XX
unspecified,11000,Unspecified,4000.0,unspecified,unspecified
travel,11001,Travel and tourism,4001.0,travel,travel
transport,11002,Transport,4002.0,transport,transport
ict,11003,ICT,4003.0,ict,ict


In [66]:
hs_mapping.tail(10)

,hs_6digit,product_id,hs_product_name_6digit,parent_id,hs_4digit,hs_2digit,hs_product_name_4digit,hs_product_name_2digit
5035,970500,10035,Collections & collectors pieces,1889.0,9705,97,Collectors pieces,Art
5036,970600,10036,Antiques,1890.0,9706,97,Antiques >100 years,Art
5037,999999,10037,"Commodities not specified, according to kind",1891.0,9999,99,Commodities not specified according to kind,Other
5038,9999AA,10038,Commodities not specified according to kind,1891.0,9999,99,Commodities not specified according to kind,Other
5039,XXXXXX,10039,Trade data discrepancies,1892.0,XXXX,XX,Trade data discrepancies,NaN
5040,unspecified,11000,Unspecified,4000.0,unspecified,unspecified,Unspecified,Unspecified
5041,travel,11001,Travel and tourism,4001.0,travel,travel,Travel and tourism,Travel and tourism
5042,transport,11002,Transport,4002.0,transport,transport,Transport,Transport
5043,ict,11003,ICT,4003.0,ict,ict,ICT,ICT
5044,financial,11004,Insurance and finance,4004.0,financial,financial,Insurance and finance,Insurance and finance


In [67]:
hs_mapping.loc[hs_mapping["hs_6digit"].isna()]

,hs_6digit,product_id,hs_product_name_6digit,parent_id,hs_4digit,hs_2digit,hs_product_name_4digit,hs_product_name_2digit


# load data to bigquery

In [70]:
job = client.load_table_from_dataframe(
    hs_mapping.set_index("hs_6digit"), "main-street-labs.company_datasets_dev.hs_mapping"
)

In [22]:
csvpath = '../../datasets/atlas_harvard/country_partner_hsproduct6digit_year_2021.csv'

In [20]:
bq_schema = [
    bigquery.SchemaField("location_id", "INTEGER"),
    bigquery.SchemaField("partner_id", "INTEGER"),
    bigquery.SchemaField("product_id", "INTEGER"),
    bigquery.SchemaField("year", "FLOAT"),
    bigquery.SchemaField("export_value", "NUMERIC"),
    bigquery.SchemaField("import_value", "NUMERIC"),
    bigquery.SchemaField("hs_eci", "FLOAT"),
    bigquery.SchemaField("hs_coi", "FLOAT"),
    bigquery.SchemaField("location_code", "STRING"),
    bigquery.SchemaField("partner_code", "STRING"),
    bigquery.SchemaField("hs_product_code", "STRING")]

In [21]:
job_config = bigquery.LoadJobConfig(
    schema=bq_schema, source_format=bigquery.SourceFormat.CSV, skip_leading_rows=1)

In [23]:
with open(csvpath, "rb") as source_file:
    job = client.load_table_from_file(
        source_file, f"main-street-labs.company_datasets_dev.country_partner_hsproduct6digit",
        job_config=job_config)
    job.result()